In [11]:
import findspark 

findspark.init()
findspark.find()

'C:/tools/spark-3.3.2-bin-hadoop3'

In [12]:
import pyspark
from pyspark.sql import SparkSession

In [13]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [14]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("spark_session") \
    .getOrCreate()

In [15]:
parsed_data = "parsed_data_daily_activity.csv"

In [16]:
spark_df = spark.read.csv(parsed_data,
                    header='true',
                    inferSchema='true',
                    ignoreLeadingWhiteSpace=True,
                    ignoreTrailingWhiteSpace=True)

In [17]:
from pyspark.sql import functions as F

In [18]:
spark_df = spark_df.withColumn("ActivityDate", F.to_date("ActivityDate", "MM/dd/yyyy"))

In [19]:
spark_df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- ActivityDate: date (nullable = true)
 |-- TotalSteps: integer (nullable = true)
 |-- TotalDistance: double (nullable = true)
 |-- TrackerDistance: double (nullable = true)
 |-- LoggedActivitiesDistance: double (nullable = true)
 |-- VeryActiveDistance: double (nullable = true)
 |-- ModeratelyActiveDistance: double (nullable = true)
 |-- LightActiveDistance: double (nullable = true)
 |-- SedentaryActiveDistance: double (nullable = true)
 |-- VeryActiveMinutes: integer (nullable = true)
 |-- FairlyActiveMinutes: integer (nullable = true)
 |-- LightlyActiveMinutes: integer (nullable = true)
 |-- SedentaryMinutes: integer (nullable = true)
 |-- Calories: integer (nullable = true)



In [51]:
from pyspark.sql.functions import avg, round, desc, col

In [45]:
result_averageByID = spark_df.groupby("Id").agg(
    round(avg("TotalSteps"), 3).alias("AverageDailySteps"),
    round(avg("TotalDistance"), 3).alias("AverageTotalDistance"),
    round(avg("Calories"), 3).alias("AverageCalories"),
).orderBy(desc("AverageDailySteps"))

In [46]:
import pandas as pd
pddf = result_averageByID.toPandas()

In [53]:
total_minutes = spark_df \
.withColumn("TotalMinutes", col("VeryActiveMinutes") + col("FairlyActiveMinutes") + col("LightlyActiveMinutes") + col("SedentaryMinutes"))

In [55]:
result_activityPercentage = total_minutes.selectExpr(
    "Id",
    "ActivityDate",
    "VeryActiveMinutes / TotalMinutes * 100 as VeryActivePercentage",
    "FairlyActiveMinutes / TotalMinutes * 100 as FairlyActivePercentage",
    "LightlyActiveMinutes / TotalMinutes * 100 as LightlyActivePercentage",
    "SedentaryMinutes / TotalMinutes * 100 as SedentaryPercentage"
)